<a href="https://colab.research.google.com/github/adityapatnaik/A-Common-NGO-Platform/blob/master/campaign_users_absolute_refresh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialize BigQuery client with service account credentials


In [8]:
from google.cloud import bigquery
from datetime import datetime

# Path to your service account JSON key file
SERVICE_ACCOUNT_FILE = "./hkm_pune_test_service_account_key.json"

# Initialize BigQuery client with service account credentials
try:
    client = bigquery.Client.from_service_account_json(SERVICE_ACCOUNT_FILE)
    print("✅ Successfully connected to Google BigQuery.")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    exit(1)  # Exit the script if the connection fails

# Define dataset ID
dataset_id = "hkm-pune-test.firestore_export"

# Test query to check connectivity
test_query = f"SELECT 1 AS test_column"
try:
    result = client.query(test_query).result()
    for row in result:
        print(f"✅ Test query successful, result: {row.test_column}")
except Exception as e:
    print(f"❌ Test query failed: {e}")
    exit(1)  # Exit if the test query fails

# Generate a timestamped backup table name
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
campaign_users_backup_table = f"{dataset_id}.campaign_users_backup_{timestamp}"

# Define original table references
campaign_users_table = f"{dataset_id}.campaign_users"
campaign_donors_table = f"{dataset_id}.campaign_donors"

print(campaign_donors_table)
print(campaign_users_table)
print(campaign_users_backup_table)

✅ Successfully connected to Google BigQuery.
✅ Test query successful, result: 1
hkm-pune-test.firestore_export.campaign_donors
hkm-pune-test.firestore_export.campaign_users
hkm-pune-test.firestore_export.campaign_users_backup_20250216_080424


# Step 1: Backup the current campaign_users table with a timestamped name

In [9]:
# Step 1: Backup the current campaign_users table with a timestamped name
backup_query = f"""
    CREATE TABLE `{campaign_users_backup_table}` AS
    SELECT * FROM `{campaign_users_table}`
"""
client.query(backup_query).result()
print(f"Backup table `{campaign_users_backup_table}` created successfully.")

Backup table `hkm-pune-test.firestore_export.campaign_users_backup_20250216_080424` created successfully.


# Step 2: Reset cumulative values to zero in the backup table

In [10]:
# Step 2: Reset cumulative values to zero in the backup table
reset_query = f"""
    UPDATE `{campaign_users_backup_table}`
    SET
        cumulative_sq_feet_raised = 0,
        cumulative_sq_feet_raised_amount = 0.0
    WHERE 1=1  -- Added a WHERE clause to update all rows.
"""
client.query(reset_query).result()
print(f"Cumulative values reset to zero in `{campaign_users_backup_table}`.")

Cumulative values reset to zero in `hkm-pune-test.firestore_export.campaign_users_backup_20250216_080424`.


# Step 3: Fetch all campaign donor entries

In [11]:
# Step 3: Fetch all campaign donor entries
donor_query = f"""
    SELECT campaign_id, donated_sq_ft, donated_sq_ft_amount
    FROM `{campaign_donors_table}`
"""
donor_rows = client.query(donor_query).result()
print(donor_rows)

# Step 4: Iterate through each donation and update cumulative values in the backup table

In [13]:
# Step 4: Iterate through each donation and update cumulative values in the backup table
for row in donor_rows:
    campaign_id = row.campaign_id
    donated_sq_ft = row.donated_sq_ft
    donated_sq_ft_amount = row.donated_sq_ft_amount

    # ✅ First, update the `campaign_id` itself
    update_self_query = f"""
        UPDATE `{campaign_users_backup_table}`
        SET
            cumulative_sq_feet_raised = IFNULL(cumulative_sq_feet_raised, 0) + @donated_sq_ft,
            cumulative_sq_feet_raised_amount = IFNULL(cumulative_sq_feet_raised_amount, 0) + @donated_sq_ft_amount
        WHERE user_id = @campaign_id
    """
    update_self_params = [
        bigquery.ScalarQueryParameter("donated_sq_ft", "INT64", donated_sq_ft),
        bigquery.ScalarQueryParameter("donated_sq_ft_amount", "NUMERIC", donated_sq_ft_amount),
        bigquery.ScalarQueryParameter("campaign_id", "INT64", campaign_id)
    ]
    client.query(update_self_query, job_config=bigquery.QueryJobConfig(
        query_parameters=update_self_params)).result()

    current_user_id = campaign_id  # Start with the campaign_id from donors table

    while current_user_id:
        # Get parent_id for the current campaign in the backup table
        parent_query = f"""
            SELECT parent_id
            FROM `{campaign_users_backup_table}`
            WHERE user_id = @current_user_id
        """
        parent_query_params = [
            bigquery.ScalarQueryParameter("current_user_id", "INT64", current_user_id)
        ]
        parent_results = client.query(parent_query, job_config=bigquery.QueryJobConfig(
            query_parameters=parent_query_params)).result()

        parent_rows = list(parent_results)
        if not parent_rows or not parent_rows[0].parent_id:
            break  # Stop if there's no parent

        current_user_id = parent_rows[0].parent_id  # Move to the parent

        # Update cumulative values for the parent in the backup table
        update_query = f"""
            UPDATE `{campaign_users_backup_table}`
            SET
                cumulative_sq_feet_raised = IFNULL(cumulative_sq_feet_raised, 0) + @donated_sq_ft,
                cumulative_sq_feet_raised_amount = IFNULL(cumulative_sq_feet_raised_amount, 0) + @donated_sq_ft_amount
            WHERE user_id = @current_user_id
        """
        update_query_params = [
            bigquery.ScalarQueryParameter("donated_sq_ft", "INT64", donated_sq_ft),
            bigquery.ScalarQueryParameter("donated_sq_ft_amount", "NUMERIC", donated_sq_ft_amount),
            bigquery.ScalarQueryParameter("current_user_id", "INT64", current_user_id)
        ]
        client.query(update_query, job_config=bigquery.QueryJobConfig(
            query_parameters=update_query_params)).result()

print(f"Cumulative updates completed successfully in `{campaign_users_backup_table}`.")

Cumulative updates completed successfully in `hkm-pune-test.firestore_export.campaign_users_backup_20250216_080424`.
